update sample metadata to correct for wrongly sorted TdTom+ cells

should be integrated in the snakemake pipeline early on

In [1]:
here::i_am("processing/1_create_seurat_rna.R")

# Load default settings
source(here::here("settings.R"))
source(here::here("utils.R"))

here() starts at /rds/project/rds-SDzz0CATGms/users/bt392/03_Stat3_RNA/code



In [2]:
args = list()
args$sce <- io$rna.sce
args$metadata <- paste0(io$basedir,"/results/rna/mapping/sample_metadata_after_mapping.txt.gz")

In [10]:
sample_metadata <- fread(args$metadata) %>%
   .[pass_rnaQC==TRUE & doublet_call==FALSE]

In [11]:
###############
## Load data ##
###############

# Load RNA expression data as SingleCellExperiment object
sce <- load_SingleCellExperiment(args$sce, cells=sample_metadata$cell, normalise = TRUE)

# Add sample metadata as colData
colData(sce) <- sample_metadata %>% tibble::column_to_rownames("cell") %>% DataFrame

In [12]:
tomato = logcounts(sce['tomato-td',])
tomato_corr = cbind(colData(sce), t(as.data.frame(as.matrix(tomato)))) %>% as.data.frame()
tomato_corr$tomato_pos = ifelse(tomato_corr$tomato>0, TRUE, FALSE)
table(tomato_corr$tdTom, tomato_corr$tomato_pos)

# annotate any cell with tomato-td expression as being tdTom+
colData(sce)$tdTom_corr = ifelse(logcounts(sce['tomato-td',]) > 0 | colData(sce)$tdTom==TRUE, TRUE, FALSE)

       
        FALSE  TRUE
  FALSE 24322   771
  TRUE   1746 18831

In [55]:
meta_new = as.data.frame(colData(sce)['tdTom_corr']) %>% tibble::rownames_to_column("cell")

sample_metadata <- fread(args$metadata) %>% .[,idx:=1:nrow(.)]

In [69]:
sample_metadata <- fread(args$metadata) %>% 
    .[,idx:=1:nrow(.)] %>%
    merge(meta_new, by='cell', all.x=TRUE) %>%
    .[order(idx),]

head(sample_metadata)

cell,sample,barcode,nFeature_RNA,nCount_RNA,mitochondrial_percent_RNA,ribosomal_percent_RNA,stage,tdTom,pass_rnaQC,doublet_score,doublet_call,celltype.mapped_mnn,celltype.score_mnn,closest.cell_mnn,idx,tdTom_corr
<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<chr>,<lgl>,<lgl>,<dbl>,<lgl>,<chr>,<dbl>,<chr>,<int>,<lgl>
SLX-21143_SITTA2_HTJH3DSX2#AAACCCAAGATGTTCC-1,SLX-21143_SITTA2_HTJH3DSX2,AAACCCAAGATGTTCC-1,4837,53199,1.47,22.87,E8.5,TRUE,TRUE,0.18,FALSE,Erythroid,1.0,cell_33109,1,TRUE
SLX-21143_SITTA2_HTJH3DSX2#AAACCCAAGCACTTTG-1,SLX-21143_SITTA2_HTJH3DSX2,AAACCCAAGCACTTTG-1,310,648,32.25,12.81,E8.5,TRUE,FALSE,NA,NA,NA,NA,NA,2,NA
SLX-21143_SITTA2_HTJH3DSX2#AAACCCATCAGACCTA-1,SLX-21143_SITTA2_HTJH3DSX2,AAACCCATCAGACCTA-1,4973,24165,0.95,19.26,E8.5,TRUE,TRUE,0.23,FALSE,Mesenchyme,1.0,ext_cell_41684,3,TRUE
SLX-21143_SITTA2_HTJH3DSX2#AAACGAAAGCGCCTTG-1,SLX-21143_SITTA2_HTJH3DSX2,AAACGAAAGCGCCTTG-1,1614,4156,6.74,28.20,E8.5,TRUE,FALSE,NA,NA,NA,NA,NA,4,NA
SLX-21143_SITTA2_HTJH3DSX2#AAACGAACAATGTTGC-1,SLX-21143_SITTA2_HTJH3DSX2,AAACGAACAATGTTGC-1,5585,34341,0.22,26.76,E8.5,TRUE,TRUE,0.22,FALSE,Anterior somitic tissues,0.8,ext_cell_66265,5,TRUE
SLX-21143_SITTA2_HTJH3DSX2#AAACGAACACACCAGC-1,SLX-21143_SITTA2_HTJH3DSX2,AAACGAACACACCAGC-1,5016,41496,1.66,25.01,E8.5,TRUE,TRUE,0.11,FALSE,Erythroid,1.0,cell_33764,6,TRUE


In [71]:
fwrite(fread(args$metadata), paste0(io$basedir,"/results/rna/mapping/sample_metadata_after_mapping_bu.txt.gz"))

In [72]:
fwrite(sample_metadata, args$metadata)